In [4]:
import argparse
import os
import sys
import math
import torch
import torch.nn as nn
import pandas as pd

from thop import profile

from utils.utils import AverageMeter

from data_providers import UCIHARDataProvider, OPPDataProvider, KUHARDataProvider, UniMiBDataProvider, WISDMDataProvider

In [5]:
pd.read_csv('harmodel.csv')

,dataset,batch,model,hardware,device,latency
0,uci,1,RTWCNN,pc,cpu,0.001637
1,uci,1,RTWCNN,pc,cpu,0.001719
2,uci,1,RTWCNN,pc,cpu,0.001655
3,uci,1,RTWCNN,pc,cpu,0.001701
4,uci,1,RTWCNN,pc,cpu,0.001667
5,uci,1,RTWCNN,pc,cpu,0.001658
6,uci,1,RTWCNN,pc,cpu,0.001674
7,uci,1,RTWCNN,pc,cpu,0.001688
8,uci,1,RTWCNN,pc,cpu,0.001636
9,uci,1,RTWCNN,pc,cpu,0.001660


# All Model

In [2]:
model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

* cpu

In [5]:
for model in model_list:
    for i in range(10):
        ! python measure_harmodel_latency.py --hardware pc --device cpu --dataset uci --arch $model --num-runs 100 --config_file harmodel.csv

RTWCNN: 0.001637
Experiment results saved to harmodel.csv
RTWCNN: 0.001719
Experiment results saved to harmodel.csv
RTWCNN: 0.001655
Experiment results saved to harmodel.csv
RTWCNN: 0.001701
Experiment results saved to harmodel.csv
RTWCNN: 0.001667
Experiment results saved to harmodel.csv
RTWCNN: 0.001658
Experiment results saved to harmodel.csv
RTWCNN: 0.001674
Experiment results saved to harmodel.csv
RTWCNN: 0.001688
Experiment results saved to harmodel.csv
RTWCNN: 0.001636
Experiment results saved to harmodel.csv
RTWCNN: 0.001660
Experiment results saved to harmodel.csv
HARLSTM: 0.001788
Experiment results saved to harmodel.csv
HARLSTM: 0.001751
Experiment results saved to harmodel.csv
HARLSTM: 0.001681
Experiment results saved to harmodel.csv
HARLSTM: 0.001724
Experiment results saved to harmodel.csv
HARLSTM: 0.001705
Experiment results saved to harmodel.csv
HARLSTM: 0.001737
Experiment results saved to harmodel.csv
HARLSTM: 0.001732
Experiment results saved to harmodel.csv
HARLSTM

* gpu

In [6]:
for model in model_list:
    for i in range(10):
        ! python measure_harmodel_latency.py --hardware pc --device gpu --dataset uci --arch $model --num-runs 100 --config_file harmodel.csv

RTWCNN: 0.183894
Experiment results saved to harmodel.csv
RTWCNN: 0.192490
Experiment results saved to harmodel.csv
RTWCNN: 0.188586
Experiment results saved to harmodel.csv
RTWCNN: 0.189613
Experiment results saved to harmodel.csv
RTWCNN: 0.195550
Experiment results saved to harmodel.csv
RTWCNN: 0.193431
Experiment results saved to harmodel.csv
RTWCNN: 0.189576
Experiment results saved to harmodel.csv
RTWCNN: 0.198902
Experiment results saved to harmodel.csv
RTWCNN: 0.182781
Experiment results saved to harmodel.csv
RTWCNN: 0.189374
Experiment results saved to harmodel.csv
HARLSTM: 0.717099
Experiment results saved to harmodel.csv
HARLSTM: 0.688467
Experiment results saved to harmodel.csv
HARLSTM: 0.709359
Experiment results saved to harmodel.csv
HARLSTM: 0.702045
Experiment results saved to harmodel.csv
HARLSTM: 0.706973
Experiment results saved to harmodel.csv
HARLSTM: 0.700059
Experiment results saved to harmodel.csv
HARLSTM: 0.767471
Experiment results saved to harmodel.csv
HARLSTM

## Model Spec

In [7]:
model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

In [8]:
for model in model_list:
    ! python measure_harmodel_metrics.py --dataset uci --arch $model --config_file harmodel_spec.csv

Flops: 672768.000000, Param size: 101286.000000MB
Flops: 0.672768, Param size: 0.101286MB
Experiment results saved to harmodel_spec.csv


Unsupported operator aten::max_pool1d encountered 3 time(s)


Flops: 768.000000, Param size: 202502.000000MB
Flops: 26.084096, Param size: 0.202502MB
Experiment results saved to harmodel_spec.csv


Unsupported operator aten::lstm encountered 1 time(s)


Flops: 1536.000000, Param size: 536070.000000MB
Flops: 68.945408, Param size: 0.536070MB
Experiment results saved to harmodel_spec.csv


...

Unsupported operator aten::lstm encountered 1 time(s)


Flops: 61539072.000000, Param size: 481542.000000MB
Flops: 61.866752, Param size: 0.481542MB
Experiment results saved to harmodel_spec.csv


Unsupported operator aten::add encountered 3 time(s)
Unsupported operator aten::adaptive_avg_pool1d encountered 1 time(s)


Flops: 34374400.000000, Param size: 269318.000000MB
Flops: 34.505472, Param size: 0.269318MB
Experiment results saved to harmodel_spec.csv


Unsupported operator aten::adaptive_avg_pool1d encountered 1 time(s)


# 결과 결합

In [9]:
import pandas as pd

In [10]:
lat_df = pd.read_csv('harmodel.csv')
spec_df = pd.read_csv('harmodel_spec.csv')

In [11]:
lat_df.head()

,dataset,batch,model,hardware,device,latency
0,uci,1,RTWCNN,pc,cpu,0.001637
1,uci,1,RTWCNN,pc,cpu,0.001719
2,uci,1,RTWCNN,pc,cpu,0.001655
3,uci,1,RTWCNN,pc,cpu,0.001701
4,uci,1,RTWCNN,pc,cpu,0.001667


In [12]:
spec_df.head()

,dataset,Model,Params(M),FLOPs(M)
0,uci,RTWCNN,0.101286,0.672768
1,uci,HARLSTM,0.202502,26.084096
2,uci,HARBiLSTM,0.536070,68.945408
3,uci,HARConvLSTM,0.295814,33.514752
4,uci,ResNetTSC,0.481542,61.866752


# calculate latency mean & std

mean

In [13]:
test = lat_df.groupby(['model', 'device', 'batch']).mean()
test

C:\Users\ai-ws\AppData\Local\Temp\ipykernel_14632\3156294001.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test = lat_df.groupby(['model', 'device', 'batch']).mean()


latency
model       device batch          
FCNTSC      cpu    1      0.001662
            gpu    1      0.151083
HARBiLSTM   cpu    1      0.001821
            gpu    1      1.432857
HARConvLSTM cpu    1      0.001756
            gpu    1      1.139593
HARLSTM     cpu    1      0.001728
            gpu    1      0.718917
RTWCNN      cpu    1      0.001670
            gpu    1      0.190420
ResNetTSC   cpu    1      0.001673
            gpu    1      1.363599

In [14]:
test = test.reset_index()
test

,model,device,batch,latency
0,FCNTSC,cpu,1,0.001662
1,FCNTSC,gpu,1,0.151083
2,HARBiLSTM,cpu,1,0.001821
3,HARBiLSTM,gpu,1,1.432857
4,HARConvLSTM,cpu,1,0.001756
5,HARConvLSTM,gpu,1,1.139593
6,HARLSTM,cpu,1,0.001728
7,HARLSTM,gpu,1,0.718917
8,RTWCNN,cpu,1,0.001670
9,RTWCNN,gpu,1,0.190420


In [15]:
temp = test[test['device'] == 'cpu'].iloc[:, [0,2]].copy()
temp.reset_index(drop=True, inplace=True)

In [16]:
temp['PC:CPU(ms)'] = test[test['device'] == 'cpu']['latency'].reset_index(drop=True)
temp['PC:GPU(ms)'] = test[test['device'] == 'gpu']['latency'].reset_index(drop=True)
temp

,model,batch,PC:CPU(ms),PC:GPU(ms)
0,FCNTSC,1,0.001662,0.151083
1,HARBiLSTM,1,0.001821,1.432857
2,HARConvLSTM,1,0.001756,1.139593
3,HARLSTM,1,0.001728,0.718917
4,RTWCNN,1,0.001670,0.190420
5,ResNetTSC,1,0.001673,1.363599


In [17]:
temp.columns = ['Model', 'Batch', 'PC:CPU(ms)', 'PC:GPU(ms)']
temp

,Model,Batch,PC:CPU(ms),PC:GPU(ms)
0,FCNTSC,1,0.001662,0.151083
1,HARBiLSTM,1,0.001821,1.432857
2,HARConvLSTM,1,0.001756,1.139593
3,HARLSTM,1,0.001728,0.718917
4,RTWCNN,1,0.001670,0.190420
5,ResNetTSC,1,0.001673,1.363599


In [18]:
result_df = pd.merge(spec_df, temp, on='Model', how='left')
result_df

,dataset,Model,Params(M),FLOPs(M),Batch,PC:CPU(ms),PC:GPU(ms)
0,uci,RTWCNN,0.101286,0.672768,1,0.001670,0.190420
1,uci,HARLSTM,0.202502,26.084096,1,0.001728,0.718917
2,uci,HARBiLSTM,0.536070,68.945408,1,0.001821,1.432857
3,uci,HARConvLSTM,0.295814,33.514752,1,0.001756,1.139593
4,uci,ResNetTSC,0.481542,61.866752,1,0.001673,1.363599
5,uci,FCNTSC,0.269318,34.505472,1,0.001662,0.151083


* save result of vision model

In [19]:
result_df.to_csv('result_harmodel.csv', index=False)